<img align="right" src="images/etcbc.png"/>

# MQL versus TF-Query

## Introduction

Bas Meeuse has written an excellent primer for exegetes.
In a series of increasingly difficult examples he shows how one can
use template based search to explore syntactic patterns in the text.

The ETCBC database makes this possible, especially by its incarnation as 
[BHSA](https://etcbc.github.io/bhsa/) data.

In that context, there are two ways to perform template based search:

* the website [SHEBANQ](https://shebanq.ancient-data.org) offers MQL
  queries that can be run and saved without installing anything;
* [Text-Fabric](https://github.com/annotation/app-bhsa)
  offers complete computational control over the data,
  including template based queries, inspired by, but different from MQL

Both MQL and TF-Query have been developed in close connection with the ETCBC, MQL by Ulrik Sandborg-Petersen, as part of his database system
[Emdros](https://emdros.org), and Text-Fabric by Dirk Roorda.
MQL is older, dating from 2005, and it is implemented in the programming language C. Text-Fabric dates from 2016 and is implemented in Python.
This time difference shows not only in various aspects of the
MQL and TF-Query syntax, but also in other aspects of usability.

In my opinion, TF-Query is more supportive of really difficult operations,
where no single query can express the issue at hand, and where the results of several queries have to be analysed and combined.

The purpose of this primer is to help people to transition from MQL to TF-Query. 
To that end we reproduce the MQL queries in the
[SHEBANQ tutorial by Bas Meeuse](https://github.com/ETCBC/Tutorials/blob/master/SHEBANQ%20tutorial%202021.pdf)
by means of Text-Fabric queries, and discuss the subtle and not so subtle
distinctions that we will encounter.

## Reference

* [MQL Query Guide](https://github.com/ETCBC/shebanq/wiki/Documents/MQL-Query-Guide.pdf) (pdf)
* [TF Query Guide](https://annotation.github.io/text-fabric/tf/about/searchusage.html) (html)
* [Other MQL-TF queries](https://nbviewer.jupyter.org/github/annotation/tutorials/blob/master/bhsa/searchFromMQL.ipynb) (notebook)

## Author
Dirk Roorda (dirk.roorda@dans.knaw.nl)

Copyright: No restrictions. Attribution appreciated. CC-BY.

# The example queries

We will compare the results of MQL queries, performed by SHEBANQ with results obtained from TF-Queries.

To that end we have downloaded the CSV results of all example queries in the tutorial by Bas. For convenience, we map the example numbers to the names of the result files and additional data of the query.

In [1]:
MQL_RESULTS = {
    1: {
        "file": "2017_q4439_text_Bas_Meeuse",
        "results": 8,
        "verses": 8,
        "words": 42,
    },
    2: {
        "file": "2017_q4440_text_Bas_Meeuse",
        "results": 156,
        "verses": 136,
        "words": 294,
    },
    "2a": {
        "file": "2017_q4467_text_Dirk_Roorda",
        "results": 160,
        "verses": 138,
        "words": 300,
    },
    3: {
        "file": "2017_q4441_text_Bas_Meeuse",
        "results": 308,
        "verses": 150,
        "words": 685,
    },
    4: {
        "file": "2017_q53_text_Wido_van Peursen",
        "results": 65,
        "verses": 51,
        "words": 315,
    },
    5: {
        "file": "2017_q494_text_Oliver_Glanz",
        "results": 23,
        "verses": 21,
        "words": 44,
    },
    6: {
        "file": "2017_q4442_text_Bas_Meeuse",
        "results": 10,
        "verses": 10,
        "words": 121,
    },
    7: {
        "file": "2017_q4334_text_Reinoud_Oosting",
        "results": 62,
        "verses": 61,
        "words": 348,
    },
    8: {
        "file": "2017_q523_text_Reinoud_Oosting",
        "results": 16,
        "verses": 16,
        "words": 32,
    },
    9: {
        "file": "2017_q491_text_Oliver_Glanz",
        "results": 344,
        "verses": 101,
        "words": 356,
    },
    "10a": {
        "file": "2017_q4416_text_Bas_Meeuse",
        "results": 232,
        "verses": 190,
        "words": 458,
    },
    "10b": {
        "file": "2017_q4437_text_Bas_Meeuse",
        "results": 2087,
        "verses": 675,
        "words": 5573,
    },
}

# Loading

We load the Text-Fabric program and the BHSA data.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from tf.app import use
from tf.core.helpers import project

from util import getTfVerses, getShebanqData, compareResults

In [4]:
VERSION = '2017'
# A = use('bhsa', hoist=globals(), version=VERSION)
A = use('bhsa:clone', checkout="clone", hoist=globals(), version=VERSION)

# Example 1

[Bas Meeuse: Example 1: Moses starts the speech](https://shebanq.ancient-data.org/hebrew/query?version=2017&id=4439)

```
[clause FOCUS
  [word lex = '>MR[' OR lex = 'DBR['] 
  [phrase function = Subj
    [word lex = 'MCH=/']
  ]
  ..
  [phrase function IN (Cmpl)
    [word lex = 'JHWH/' OR lex = '>LHJM/']
  ]
]

```

In [5]:
(verses, words) = getShebanqData(A, MQL_RESULTS, 1)

8 results in 8 verses with 42 words


In [6]:
query = """
clause
  word lex=>MR[|DBR[
  <: phrase function=Subj
    word lex=MCH=/
  < phrase function=Cmpl
    word lex=JHWH/|>LHJM/
"""

In [7]:
results = A.search(query)

  1.68s 8 results


We need to do some effort to count the distinct verses and focused words in the results.
The function `getTfVerse()`, imported from the adhoc `utils` module in this directory does it.

In [8]:
(tfVerses, tfWords) = getTfVerses(A, results, (0,))

  8 verses
 42 words


**The numbers agree exactly**.

This is not a rigorous proof that the MQL results are identical to the TF results, but it is a very good indication.

However, we will also check that all words and verses in the shebanq results are
exactly the words and verses in the TF results. We try to find the first difference.

In [9]:
compareResults(A, verses, words, tfVerses, tfWords)

VERSES EQUAL
WORDS EQUAL


Finally, here are the results of the TF query.

In [10]:
A.table(results)

n,p,clause,word,phrase,word,phrase,word
1,Exodus 3:11,וַיֹּ֤אמֶר מֹשֶׁה֙ אֶל־הָ֣אֱלֹהִ֔ים,יֹּ֤אמֶר,מֹשֶׁה֙,מֹשֶׁה֙,אֶל־הָ֣אֱלֹהִ֔ים,אֱלֹהִ֔ים
2,Exodus 3:13,וַיֹּ֨אמֶר מֹשֶׁ֜ה אֶל־הָֽאֱלֹהִ֗ים,יֹּ֨אמֶר,מֹשֶׁ֜ה,מֹשֶׁ֜ה,אֶל־הָֽאֱלֹהִ֗ים,אֱלֹהִ֗ים
3,Exodus 4:10,וַיֹּ֨אמֶר מֹשֶׁ֣ה אֶל־יְהוָה֮,יֹּ֨אמֶר,מֹשֶׁ֣ה,מֹשֶׁ֣ה,אֶל־יְהוָה֮,יְהוָה֮
4,Exodus 19:23,וַיֹּ֤אמֶר מֹשֶׁה֙ אֶל־יְהוָ֔ה,יֹּ֤אמֶר,מֹשֶׁה֙,מֹשֶׁה֙,אֶל־יְהוָ֔ה,יְהוָ֔ה
5,Exodus 33:12,וַיֹּ֨אמֶר מֹשֶׁ֜ה אֶל־יְהוָ֗ה,יֹּ֨אמֶר,מֹשֶׁ֜ה,מֹשֶׁ֜ה,אֶל־יְהוָ֗ה,יְהוָ֗ה
6,Numbers 11:11,וַיֹּ֨אמֶר מֹשֶׁ֜ה אֶל־יְהוָ֗ה,יֹּ֨אמֶר,מֹשֶׁ֜ה,מֹשֶׁ֜ה,אֶל־יְהוָ֗ה,יְהוָ֗ה
7,Numbers 14:13,וַיֹּ֥אמֶר מֹשֶׁ֖ה אֶל־יְהוָ֑ה,יֹּ֥אמֶר,מֹשֶׁ֖ה,מֹשֶׁ֖ה,אֶל־יְהוָ֑ה,יְהוָ֑ה
8,Numbers 27:15,וַיְדַבֵּ֣ר מֹשֶׁ֔ה אֶל־יְהוָ֖ה,יְדַבֵּ֣ר,מֹשֶׁ֔ה,מֹשֶׁ֔ה,אֶל־יְהוָ֖ה,יְהוָ֖ה


# Example 2

[Bas Meeuse: Example 2: FJM + prep. L](https://shebanq.ancient-data.org/hebrew/query?version=2017&id=4440)

```
[clause
  [word FOCUS lex = 'FJM[']
  ..
  [word FOCUS lex = "L"]
  [word lex <> '<JN/' AND lex <> 'PNH/']
]
```

In [11]:
(verses, words) = getShebanqData(A, MQL_RESULTS, 2)

156 results in 136 verses with 294 words


In [12]:
query = """
clause
  word lex=FJM[
  < word lex=L
  <: word lex#<JN/|PNH/
"""

In [13]:
results = A.search(query)

  1.61s 155 results


**N.B.:** one result less than in SHEBANQ.

In [14]:
(tfVerses, tfWords) = getTfVerses(A, results, (1, 2))

135 verses
292 words


In [15]:
compareResults(A, verses, words, tfVerses, tfWords)

DIFFERENCE:
('Joshua', 8, 2)
('Joshua', 8, 12)
DIFFERENCE:
116852 = FIJM
117091 = J.@63FEM


-1

The TF results are skipping Joshua 8:2!

We expand this verse in SHEBANQ:

![josh](images/josh.png)

That there is a gap in the clause, right after the word `L` between words 116853 and 116858.
In MQL, the adjacency of things is relative to the container it is in.
If the container has a gap, the words around the gap are considered adjacent.

In this example it means that this part of the query:

```
  [word FOCUS lex = "L"]
  [word lex <> '<JN/' AND lex <> 'PNH/']
```

is matched by words 116853 and **116858**.
And the MQL query considers those two words as adjacent *within the clause*.

In Text-Fabric, adjacency between words is absolute: it is not relative to a container object.

So this part of the query

```
  < word lex=L
  <: word lex#<JN/|PNH/
```

is *not* matched by words 116853 and **116854**, because 116854 is not part of the embedding clause.

The Text-Fabric notion of adjacency is more crude. 
The reason is that in Text-Fabric, the query does not have to be a tree, where each object has a unique
immediate parent object. There could be several parent objects, and each of the parents may have different
gaps, and if we had the concept of relative adjacency, our query language would need a way to express that.

It has not, and to me it is an open question whether we should complicate search templates in that way.

Anyway, as it stands,  there is no obvious workaround to get the exactly the same behaviour as the MQL query.

That said, we can try something that comes close:

We state that the `L` is is not immediately followed by a word that is `<JN/` or `PNH/`.

In [16]:
query = """
clause
  word lex=FJM[
  < word lex=L
  /without/
  <: w3:word lex=<JN/|PNH/
  /-/
"""

In [17]:
results2 = A.search(query)

  0.80s 161 results


It turns out that we get more results than in SHEBANQ.
We first count the verses and words involved in the results.

In [18]:
(tfVerses2, tfWords2) = getTfVerses(A, results2, (1, 2))

139 verses
302 words


In [19]:
compareResults(A, verses, words, tfVerses2, tfWords2)

DIFFERENCE:
('Genesis', 43, 32)
('Genesis', 27, 37)
DIFFERENCE:
24760 = J.@FI71JMW.
14288 = FAM:T.I71JW


-1

Now the situation is reversed: Genesis 27:37 is in the TF results, but skipped by SHEBANQ.

In [20]:
A.table(results2, start=3, end=3)

n,p,clause,word,word
3,Genesis 27:37,הֵ֣ן גְּבִ֞יר שַׂמְתִּ֥יו לָךְ֙,שַׂמְתִּ֥יו,לָךְ֙


The Genesis 27:37 result has something in common with the Joshua 8:2 result in SHEBANQ that we saw above: the `L` has a pronominal suffix.

Here it is also the last word in the clause.
So it seems to be an intended result of the MQL query.

Let's make a mental shift: what *is* the intention of the MQL query?
Here is a bit of query-exegesis, in that the query itself is the object of the exegesis.

The MQL query mentions three `[word]` objects, but it puts only the first two of them in `FOCUS`. 

1. it is not interested in the actual value of the third one;
2. the third `[word]` is constrained by a very loose restriction: it can be anything, except two specific values.

These two things point to the intended meaning of the query, namely:

> find a clause with the word `FJM[`, and somewhere after that the word `L`, 
which is not followed by either the word `<JN/` or the word `PNH/`.

This differs subtly from what the query actually says:

> find a clause with the word `FJM[`, and somewhere after that the word `L`, 
which is followed by another word that is not `<JN/` and not `PNH/`.

The difference is one of *quantification*.

More schematically, the MQL states literally:

> there is a **word** after **a** that is not **b** and not **c**

But the intention is:

> for each **word** after **a** it is not **b** and not **c**

MQL also has a concept of quantifier, a bit more limited than in TF: `NOTEXIST`.
Let's try it:

```
[clause
  [word FOCUS lex = 'FJM[']
  ..
  [word FOCUS lex = "L"]
  NOTEXIST [word lex = '<JN/' OR lex = 'PNH/']
]

```

See 
[Dirk Roorda: Example 2: not exist](https://shebanq.ancient-data.org/hebrew/query?version=2017&id=4467)

In [21]:
(verses2, words2) = getShebanqData(A, MQL_RESULTS, "2a")

160 results in 138 verses with 300 words


Now we have one result more in Text-Fabric than in SHEBANQ.

In [22]:
compareResults(A, verses2, words2, tfVerses2, tfWords2)

DIFFERENCE:
('2_Samuel', 23, 5)
('2_Samuel', 14, 7)
DIFFERENCE:
174875 = F@74M
168181 = *FWM


-1

2 Samuel 14:7 is skipped by SHEBANQ.

In [23]:
A.table(results2, start=58, end=58)

n,p,clause,word,word
58,2_Samuel 14:7,לְבִלְתִּ֧י שִׂים־לְאִישִׁ֛י שֵׁ֥ם וּשְׁאֵרִ֖ית עַל־פְּנֵ֥י הָאֲדָמָֽה׃ פ,שִׂים־,לְ


When we look it up in SHEBANQ we find this:

![sam](images/sam.png)

The thing here is that word 168188 is `PNH/`.
It turns out that the `NOTEXIST` operator in MQL quantifies over all words that *follow* from that position.

If `NOTEXIST [word properties]` meant that there is no word *at* that position with those properties, all was well for our purposes.
But it means that there is no word *from* that position with those properties.

So ot turns out: nice idea, but it does not work out in MQL.

Now the tide has turned: we have trouble in MQL to find a query that exactly matches our intention, while in TF we can.

Still, there might be problems.

If there is a clause, with `L`, then a gap, and then either `<JN/` or `PNH/`,
the SHEBANQ query would skip it, but the Text-Fabric query would include it.

Let's check in Text-Fabric whether this occurs.

In [24]:
query = """
clause
  clause_atom
    word lex=L
    :=
  < clause_atom
    =: word lex=<JN/|PNH/
"""

In [25]:
results = A.search(query)

  0.88s 0 results


Nope.

But is this query itself right?
Let's look for a known case, namely Joshua 8:2 above.

In [26]:
query = """
clause
  clause_atom
    word lex=L
    :=
  < clause_atom
    =: word lex=MN
"""

In [27]:
results = A.search(query)

  0.90s 2 results


In [28]:
A.table(results)

n,p,clause,clause_atom,word,clause_atom,word
1,Joshua 8:2,שִׂים־לְךָ֥ מֵאַחֲרֶֽיהָ׃,שִׂים־לְךָ֥,לְךָ֥,מֵאַחֲרֶֽיהָ׃,מֵ
2,Hosea 10:15,כָּ֗כָה עָשָׂ֤ה לָכֶם֙ מִפְּנֵ֖י רָעַ֣ת רָֽעַתְכֶ֑ם,כָּ֗כָה עָשָׂ֤ה לָכֶם֙,לָכֶם֙,מִפְּנֵ֖י רָעַ֣ת רָֽעַתְכֶ֑ם,מִ


Yes, this kind of query finds exactly what we are looking for.

**Conclusion**

In Text-Fabric we have found a query with slightly different results.
But these results match the intention of the query just a bit better than the original query.

We tried to improve the MQL query by using `NOTEXIST`, but that did not work out.

However, the TF query might include (contrived) cases that the MQL query would rightfully skip. 
We can verify whether those cases actually exist by running a separate TF query, and it turns out they do not exist.

**Lesson**

Whenever an exegesis hinges on the results of a query, check and double check.
You probably will have to run multiple queries in SHEBANQ and combine the results.
This will quickly get very cumbersome.
If that happens, it starts to pay off to use Text-Fabric, where you have more complete power over 
the computations and their results.

# Example 3

[Bas Meeuse: Example 3: Whose people?](https://shebanq.ancient-data.org/hebrew/query?version=2017&id=4441)

```
[phrase_atom FOCUS
  [word AS P sp = prps]
  ..
  [word lex = "W" OR lex = ">W"]
  ..
  [word prs !~ "a" AND prs_ps = P.ps]
]
```

In [29]:
(verses, words) = getShebanqData(A, MQL_RESULTS, 3)

308 results in 150 verses with 685 words


In [30]:
query = """
phrase_atom
  p:word sp=prps
  < word lex=W|>W
  < w:word prs#a
  
p .ps=prs_ps. w
"""

In [31]:
results = A.search(query)

  1.65s 308 results


In [32]:
(tfVerses, tfWords) = getTfVerses(A, results, (0,))

150 verses
685 words


In [33]:
compareResults(A, verses, words, tfVerses, tfWords)

VERSES EQUAL
WORDS EQUAL


In [34]:
A.table(results, end=5)

n,p,phrase_atom,word,word,word
1,Genesis 6:18,אַתָּ֕ה וּבָנֶ֛יךָ וְאִשְׁתְּךָ֥ וּנְשֵֽׁי־בָנֶ֖יךָ,אַתָּ֕ה,וּ,בָנֶ֛יךָ
2,Genesis 6:18,אַתָּ֕ה וּבָנֶ֛יךָ וְאִשְׁתְּךָ֥ וּנְשֵֽׁי־בָנֶ֖יךָ,אַתָּ֕ה,וּ,אִשְׁתְּךָ֥
3,Genesis 6:18,אַתָּ֕ה וּבָנֶ֛יךָ וְאִשְׁתְּךָ֥ וּנְשֵֽׁי־בָנֶ֖יךָ,אַתָּ֕ה,וּ,בָנֶ֖יךָ
4,Genesis 6:18,אַתָּ֕ה וּבָנֶ֛יךָ וְאִשְׁתְּךָ֥ וּנְשֵֽׁי־בָנֶ֖יךָ,אַתָּ֕ה,וְ,אִשְׁתְּךָ֥
5,Genesis 6:18,אַתָּ֕ה וּבָנֶ֛יךָ וְאִשְׁתְּךָ֥ וּנְשֵֽׁי־בָנֶ֖יךָ,אַתָּ֕ה,וְ,בָנֶ֖יךָ


# Example 4

[Wido van Peursen: Judges 5.1 (Sample query)](https://shebanq.ancient-data.org/hebrew/query?version=2017&id=53)

```
[clause
  [phrase FOCUS function=Pred
    [word sp=verb AND nu=sg AND gn=f]
  ]
  ..
  [phrase FOCUS function=Subj
    [word sp=conj]
  ]
]
```

In [35]:
(verses, words) = getShebanqData(A, MQL_RESULTS, 4)

65 results in 51 verses with 315 words


In [36]:
query = """
clause
  phrase function=Pred
    word sp=verb nu=sg gn=f
  < phrase function=Subj
    word sp=conj
"""

In [37]:
results = A.search(query)

  1.43s 65 results


In [38]:
(tfVerses, tfWords) = getTfVerses(A, results, (1, 3))

 51 verses
315 words


In [39]:
compareResults(A, verses, words, tfVerses, tfWords)

VERSES EQUAL
WORDS EQUAL


In [40]:
A.table(results, end=5)

n,p,clause,phrase,word,phrase,word
1,Genesis 24:61,וַתָּ֨קָם רִבְקָ֜ה וְנַעֲרֹתֶ֗יהָ,תָּ֨קָם,תָּ֨קָם,רִבְקָ֜ה וְנַעֲרֹתֶ֗יהָ,וְ
2,Genesis 31:14,וַתַּ֤עַן רָחֵל֙ וְלֵאָ֔ה,תַּ֤עַן,תַּ֤עַן,רָחֵל֙ וְלֵאָ֔ה,וְ
3,Genesis 33:7,וַתִּגַּ֧שׁ גַּם־לֵאָ֛ה וִילָדֶ֖יהָ,תִּגַּ֧שׁ,תִּגַּ֧שׁ,גַּם־לֵאָ֛ה וִילָדֶ֖יהָ,וִ
4,Genesis 47:13,וַתֵּ֜לַהּ אֶ֤רֶץ מִצְרַ֨יִם֙ וְאֶ֣רֶץ כְּנַ֔עַן מִפְּנֵ֖י הָרָעָֽב׃,תֵּ֜לַהּ,תֵּ֜לַהּ,אֶ֤רֶץ מִצְרַ֨יִם֙ וְאֶ֣רֶץ כְּנַ֔עַן,וְ
5,Exodus 15:16,תִּפֹּ֨ל עֲלֵיהֶ֤ם אֵימָ֨תָה֙ וָפַ֔חַד,תִּפֹּ֨ל,תִּפֹּ֨ל,אֵימָ֨תָה֙ וָפַ֔חַד,וָ


# Example 5

[Oliver Glanz: DHQ article: taking a woman](https://shebanq.ancient-data.org/hebrew/query?version=2017&id=494)

```
[clause
  [phrase FOCUS function IN (Pred,PreC)
    [word lex = "LQX["]
  ]
  ..
  [phrase function = Objc
    [word FOCUS gn =  f AND nu=sg AND sp=nmpr]
  ]
]
```

In [41]:
(verses, words) = getShebanqData(A, MQL_RESULTS, 5)

23 results in 21 verses with 44 words


In [42]:
query = """
clause
  phrase function=Pred|PreC
    word lex=LQX[
  < phrase function=Objc
    word gn=f nu=sg sp=nmpr
"""

In [43]:
results = A.search(query)

  1.35s 23 results


In [44]:
(tfVerses, tfWords) = getTfVerses(A, results, (2, 4))

 21 verses
 44 words


In [45]:
compareResults(A, verses, words, tfVerses, tfWords)

VERSES EQUAL
WORDS EQUAL


In [46]:
A.table(results, end=5)

n,p,clause,phrase,word,phrase,word
1,Genesis 11:31,וַיִּקַּ֨ח תֶּ֜רַח אֶת־אַבְרָ֣ם בְּנֹ֗ו וְאֶת־לֹ֤וט בֶּן־הָרָן֙ בֶּן־בְּנֹ֔ו וְאֵת֙ שָׂרַ֣י כַּלָּתֹ֔ו אֵ֖שֶׁת אַבְרָ֣ם בְּנֹ֑ו,יִּקַּ֨ח,יִּקַּ֨ח,אֶת־אַבְרָ֣ם בְּנֹ֗ו וְאֶת־לֹ֤וט בֶּן־הָרָן֙ בֶּן־בְּנֹ֔ו וְאֵת֙ שָׂרַ֣י כַּלָּתֹ֔ו אֵ֖שֶׁת אַבְרָ֣ם בְּנֹ֑ו,שָׂרַ֣י
2,Genesis 12:5,וַיִּקַּ֣ח אַבְרָם֩ אֶת־שָׂרַ֨י אִשְׁתֹּ֜ו וְאֶת־לֹ֣וט בֶּן־אָחִ֗יו וְאֶת־כָּל־רְכוּשָׁם֙ וְאֶת־הַנֶּ֖פֶשׁ,יִּקַּ֣ח,יִּקַּ֣ח,אֶת־שָׂרַ֨י אִשְׁתֹּ֜ו וְאֶת־לֹ֣וט בֶּן־אָחִ֗יו וְאֶת־כָּל־רְכוּשָׁם֙ וְאֶת־הַנֶּ֖פֶשׁ,שָׂרַ֨י
3,Genesis 16:3,וַתִּקַּ֞ח שָׂרַ֣י אֵֽשֶׁת־אַבְרָ֗ם אֶת־הָגָ֤ר הַמִּצְרִית֙ שִׁפְחָתָ֔הּ מִקֵּץ֙ עֶ֣שֶׂר שָׁנִ֔ים,תִּקַּ֞ח,תִּקַּ֞ח,אֶת־הָגָ֤ר הַמִּצְרִית֙ שִׁפְחָתָ֔הּ,הָגָ֤ר
4,Genesis 20:2,וַיִּקַּ֖ח אֶת־שָׂרָֽה׃,יִּקַּ֖ח,יִּקַּ֖ח,אֶת־שָׂרָֽה׃,שָׂרָֽה׃
5,Genesis 24:61,וַיִּקַּ֥ח הָעֶ֛בֶד אֶת־רִבְקָ֖ה,יִּקַּ֥ח,יִּקַּ֥ח,אֶת־רִבְקָ֖ה,רִבְקָ֖ה


# Example 6

[Bas Meeuse: Example 6: Turning point in Deut. 29,3?](https://shebanq.ancient-data.org/hebrew/query?version=2017&id=4442)

```
[clause focus
  [phrase function = Nega]
  [phrase typ = VP
    [word ps = p3]
  ]
  [phrase function = Subj]
  ..
  [phrase function = Time
    [word first lex = "<D"]
  ]
]

```

In [47]:
(verses, words) = getShebanqData(A, MQL_RESULTS, 6)

10 results in 10 verses with 121 words


In [48]:
query = """
clause
  phrase function=Nega
  <: phrase typ=VP
    word ps=p3
  <: phrase function=Subj
  < phrase function=Time
    =: word lex=<D
"""

In [49]:
results = A.search(query)

  1.87s 10 results


In [50]:
(tfVerses, tfWords) = getTfVerses(A, results, (0,))

 10 verses
121 words


In [51]:
compareResults(A, verses, words, tfVerses, tfWords)

VERSES EQUAL
WORDS EQUAL


In [52]:
A.table(results, end=5)

n,p,clause,phrase,phrase,word,phrase,phrase,word
1,Genesis 32:33,עַל־כֵּ֡ן לֹֽא־יֹאכְל֨וּ בְנֵֽי־יִשְׂרָאֵ֜ל אֶת־גִּ֣יד הַנָּשֶׁ֗ה עַ֖ד הַיֹּ֣ום הַזֶּ֑ה,לֹֽא־,יֹאכְל֨וּ,יֹאכְל֨וּ,בְנֵֽי־יִשְׂרָאֵ֜ל,עַ֖ד הַיֹּ֣ום הַזֶּ֑ה,עַ֖ד
2,Exodus 10:6,אֲשֶׁ֨ר לֹֽא־רָא֤וּ אֲבֹתֶ֨יךָ֙ וַאֲבֹו֣ת אֲבֹתֶ֔יךָ מִיֹּ֗ום עַ֖ד הַיֹּ֣ום הַזֶּ֑ה,לֹֽא־,רָא֤וּ,רָא֤וּ,אֲבֹתֶ֨יךָ֙ וַאֲבֹו֣ת אֲבֹתֶ֔יךָ,עַ֖ד הַיֹּ֣ום הַזֶּ֑ה,עַ֖ד
3,Leviticus 19:13,לֹֽא־תָלִ֞ין פְּעֻלַּ֥ת שָׂכִ֛יר אִתְּךָ֖ עַד־בֹּֽקֶר׃,לֹֽא־,תָלִ֞ין,תָלִ֞ין,פְּעֻלַּ֥ת שָׂכִ֛יר,עַד־בֹּֽקֶר׃,עַד־
4,Deuteronomy 29:3,וְלֹֽא־נָתַן֩ יְהוָ֨ה לָכֶ֥ם לֵב֙ וְעֵינַ֥יִם וְאָזְנַ֣יִם עַ֖ד הַיֹּ֥ום הַזֶּֽה׃,לֹֽא־,נָתַן֩,נָתַן֩,יְהוָ֨ה,עַ֖ד הַיֹּ֥ום הַזֶּֽה׃,עַ֖ד
5,Deuteronomy 34:6,וְלֹֽא־יָדַ֥ע אִישׁ֙ אֶת־קְבֻ֣רָתֹ֔ו עַ֖ד הַיֹּ֥ום הַזֶּֽה׃,לֹֽא־,יָדַ֥ע,יָדַ֥ע,אִישׁ֙,עַ֖ד הַיֹּ֥ום הַזֶּֽה׃,עַ֖ד


# Example 7

[Reinoud Oosting: verb שׂים ('to set/place') with double object](https://shebanq.ancient-data.org/hebrew/query?version=2017&id=4334)

```
[clause FOCUS
  [phrase function IN (PreO, PtcO)
     [word sp = verb AND vs = qal AND lex = "FJM[" ]
     ]
   ..
  [phrase function = Objc ]
]
OR
[clause FOCUS
   [phrase function = Objc ]
   ..
   [phrase function IN (PreO, PtcO)
     [word sp = verb AND vs = qal AND lex = "FJM[" ]
   ]
]
OR
[clause FOCUS
   [phrase typ = VP AND NOT function IN(PreO, PtcO)
      [word sp = verb AND vs = qal AND lex = "FJM[" ]
   ]
   ..
   [phrase function = Objc ]
   ..
   [phrase function = Objc ]
]
OR
[clause FOCUS
  [phrase function = Objc ]
  ..
  [phrase typ = VP AND NOT function IN (PreO, PtcO)
      [word sp = verb AND vs = qal AND lex = "FJM[" ]
  ]
  ..
  [phrase function = Objc ]
]
OR
[clause FOCUS
  [phrase function = Objc ]
  ..
  [phrase function = Objc ]
  ..
  [phrase typ = VP AND NOT function IN (PreO, PtcO)
      [word sp = verb AND vs = qal AND lex = "FJM[" ]
  ]
]
```

In [53]:
(verses, words) = getShebanqData(A, MQL_RESULTS, 7)

62 results in 61 verses with 348 words


There is no `OR` in TF-Query.
Instead, we run a separate query for each alternative and combine the results.

However, we can rewrite the first two alternatives into one query.
Note that they both specify a clause in which 2 phrases of a certain type occur.
The two alternatives specify the two different orders in which these phrases occur.
In TF-query there is no implicit order between the members of a template,
so we do not have to separate cases.
In MQL there is the `UNORDERED GROUP` construction with the same effect, which Reinoud could have used.

The same holds for alternatives 3, 4, 5, which are merely order variants of 3 phrases within a clause.

In TF we have to stipulate that the two `Objc` phrases are not the same one,
because in TF-Query it is not assumed that the different blocks should be matched by
different parts in the text.

We could do that by means of the inequality operator:

```
  phrase function=Objc
  # phrase function=Objc
```

but that will duplicate the results, because if phrase1, phrase2 is a result, then phrase2, phrase1 is also a result.

So it is better to stipulate that one of them comes before the other:

```
  phrase function=Objc
  < phrase function=Objc
```

In [54]:
query1= """
clause
  phrase function=PreO|PtcO
    word sp=verb vs=qal lex=FJM[
  phrase function=Objc
"""

query2 = """
clause
  phrase typ=VP function#PreO|PtcO
    word sp=verb vs=qal lex=FJM[
  phrase function=Objc
  < phrase function=Objc
"""

In [55]:
results1 = A.search(query1)
results2 = A.search(query2)

  0.91s 21 results
  1.35s 41 results


The number of results nicely add up to the expected 62.

In [56]:
(tfVerses1, tfWords1) = getTfVerses(A, results1, (0,))
(tfVerses2, tfWords2) = getTfVerses(A, results2, (0,))

 21 verses
108 words
 40 verses
240 words


We combine the verses and words and test for equality.

In [57]:
tfVerses = sorted(set(tfVerses1) | set(tfVerses2))
tfWords = sorted(set(tfWords1) | set(tfWords2))

In [58]:
compareResults(A, verses, words, tfVerses, tfWords)

VERSES EQUAL
WORDS EQUAL


In [59]:
A.table(results1, end=5)
A.table(results2, end=5)

n,p,clause,phrase,word,phrase
1,Genesis 27:37,הֵ֣ן גְּבִ֞יר שַׂמְתִּ֥יו לָךְ֙,שַׂמְתִּ֥יו,שַׂמְתִּ֥יו,גְּבִ֞יר
2,Genesis 47:6,וְשַׂמְתָּ֛ם שָׂרֵ֥י מִקְנֶ֖ה,שַׂמְתָּ֛ם,שַׂמְתָּ֛ם,שָׂרֵ֥י מִקְנֶ֖ה
3,Joshua 8:28,וַיְשִׂימֶ֤הָ תֵּל־עֹולָם֙ שְׁמָמָ֔ה עַ֖ד הַיֹּ֥ום הַזֶּֽה׃,יְשִׂימֶ֤הָ,יְשִׂימֶ֤הָ,תֵּל־עֹולָם֙ שְׁמָמָ֔ה
4,1_Samuel 11:2,וְשַׂמְתִּ֥יהָ חֶרְפָּ֖ה עַל־כָּל־יִשְׂרָאֵֽל׃,שַׂמְתִּ֥יהָ,שַׂמְתִּ֥יהָ,חֶרְפָּ֖ה
5,1_Samuel 18:13,וַיְשִׂמֵ֥הוּ לֹ֖ו שַׂר־אָ֑לֶף,יְשִׂמֵ֥הוּ,יְשִׂמֵ֥הוּ,שַׂר־אָ֑לֶף


n,p,clause,phrase,word,phrase,phrase
1,Genesis 28:18,וַיָּ֥שֶׂם אֹתָ֖הּ מַצֵּבָ֑ה,יָּ֥שֶׂם,יָּ֥שֶׂם,אֹתָ֖הּ,מַצֵּבָ֑ה
2,Exodus 28:12,וְשַׂמְתָּ֞ אֶת־שְׁתֵּ֣י הָאֲבָנִ֗ים עַ֚ל כִּתְפֹ֣ת הָֽאֵפֹ֔ד אַבְנֵ֥י זִכָּרֹ֖ן לִבְנֵ֣י יִשְׂרָאֵ֑ל,שַׂמְתָּ֞,שַׂמְתָּ֞,אֶת־שְׁתֵּ֣י הָאֲבָנִ֗ים,אַבְנֵ֥י זִכָּרֹ֖ן לִבְנֵ֣י יִשְׂרָאֵ֑ל
3,Leviticus 24:6,וְשַׂמְתָּ֥ אֹותָ֛ם שְׁתַּ֥יִם מַֽעֲרָכֹ֖ות עַ֛ל הַשֻּׁלְחָ֥ן הַטָּהֹ֖ר לִפְנֵ֥י יְהוָֽה׃,שַׂמְתָּ֥,שַׂמְתָּ֥,אֹותָ֛ם,שְׁתַּ֥יִם מַֽעֲרָכֹ֖ות
4,Joshua 8:12,וַיָּ֨שֶׂם אֹותָ֜ם אֹרֵ֗ב בֵּ֧ין בֵּֽית־אֵ֛ל וּבֵ֥ין הָעַ֖י מִיָּ֥ם לָעִֽיר׃,יָּ֨שֶׂם,יָּ֨שֶׂם,אֹותָ֜ם,אֹרֵ֗ב
5,Judges 8:31,וַיָּ֥שֶׂם אֶת־שְׁמֹ֖ו אֲבִימֶֽלֶךְ׃,יָּ֥שֶׂם,יָּ֥שֶׂם,אֶת־שְׁמֹ֖ו,אֲבִימֶֽלֶךְ׃


# Example 8

[Reinoud Oosting: to withhold + from](https://shebanq.ancient-data.org/hebrew/query?version=2017&id=523)

```
[clause
  [phrase function = Pred OR function = PreC
    [word FOCUS sp = verb AND vs = qal AND lex = "MN<[" ]
  ]
  ..
  [phrase function = Cmpl
    [word FOCUS sp = prep ]
  ]
]
OR
[clause
  [phrase function = Cmpl
    [word FOCUS sp = prep ]
  ]
  ..
  [phrase function = Pred OR function = PreC
    [word FOCUS sp = verb AND vs = qal AND lex = "MN<["]
  ]
]
```

In [60]:
(verses, words) = getShebanqData(A, MQL_RESULTS, 8)

16 results in 16 verses with 32 words


Note, that as in example 7, the `OR` is used to specify different orders of the same objects.
Since in TF-Query the order is not important, by default, we do not need the `OR`, leading to a much simpler query.

In [61]:
query = """
clause
  phrase function=Pred|PreC
    word sp=verb vs=qal lex=MN<[
  phrase function=Cmpl
    word sp=prep
"""

In [62]:
results = A.search(query)

  1.37s 16 results


In [63]:
(tfVerses, tfWords) = getTfVerses(A, results, (2, 4))

 16 verses
 32 words


In [64]:
compareResults(A, verses, words, tfVerses, tfWords)

VERSES EQUAL
WORDS EQUAL


In [65]:
A.table(results, end=5)

n,p,clause,phrase,word,phrase,word
1,Genesis 30:2,אֲשֶׁר־מָנַ֥ע מִמֵּ֖ךְ פְּרִי־בָֽטֶן׃,מָנַ֥ע,מָנַ֥ע,מִמֵּ֖ךְ,מִמֵּ֖ךְ
2,1_Kings 20:7,וְלֹ֥א מָנַ֖עְתִּי מִמֶּֽנּוּ׃,מָנַ֖עְתִּי,מָנַ֖עְתִּי,מִמֶּֽנּוּ׃,מִמֶּֽנּוּ׃
3,Jeremiah 2:25,מִנְעִ֤י רַגְלֵךְ֙ מִיָּחֵ֔ף,מִנְעִ֤י,מִנְעִ֤י,מִיָּחֵ֔ף,מִ
4,Jeremiah 5:25,וְחַטֹּ֣אותֵיכֶ֔ם מָנְע֥וּ הַטֹּ֖וב מִכֶּֽם׃,מָנְע֥וּ,מָנְע֥וּ,מִכֶּֽם׃,מִכֶּֽם׃
5,Jeremiah 31:16,מִנְעִ֤י קֹולֵךְ֙ מִבֶּ֔כִי,מִנְעִ֤י,מִנְעִ֤י,מִבֶּ֔כִי,מִ


# Example 9

[Oliver Glanz: DHQ article: discourse pattern deviation](https://shebanq.ancient-data.org/hebrew/query?version=2017&id=491)

```
[[clause domain = "N"
  [phrase function = Pred
    [word
      [word lex = "DBR["]
        OR
        [word lex = ">MR["]
        OR
        [word lex = "QR>["]
      ]
  ]
    ..
    [phrase FOCUS function = Subj
      [word AS samesubject]
    ]
    ..
    [phrase FOCUS function = Cmpl
      [word AS samecomplement]
    ]
  ]
  [clause domain = "N"]* {0-1}
  [clause domain = "Q"]* {1-50}
  [clause domain = "N"
    [phrase function = Pred
      [word
        [word lex = "DBR["]
        OR
        [word lex = ">MR["]
        OR
        [word lex = "QR>["]
      ]
    ]
    ..
    [phrase FOCUS function = Subj
      [word lex = samesubject.lex]
    ]
    ..
    [phrase FOCUS function = Cmpl
      [word lex = samecomplement.lex]
    ]
  ]]
OR
  [[clause domain = "N"
    [phrase function = Pred
      [word
        [word lex = "DBR["]
        OR
        [word lex = ">MR["]
        OR
        [word lex = "QR>["]
      ]
    ]
    ..
    [phrase FOCUS function = Cmpl
      [word AS samecomplement2]
    ]
    ..
    [phrase FOCUS function = Subj
      [word AS samesubject2]
    ]
  ]
  [clause domain = "N"]* {0-1}
  [clause domain = "Q"]* {1-50}
  [clause domain = "N"
    [phrase function = Pred
      [word
        [word lex = "DBR["]
        OR
        [word lex = ">MR["]
        OR
        [word lex = "QR>["]
      ]
    ]
    ..
    [phrase FOCUS function = Cmpl
      [word lex = samecomplement2.lex]
    ]
    ..
    [phrase FOCUS function = Subj
      [word lex = samesubject2.lex]
    ]
  ]]
  OR
  [[clause domain = "N"
    [phrase function = Pred
      [word
        [word lex = "DBR["]
        OR
        [word lex = ">MR["]
        OR
        [word lex = "QR>["]
      ]
    ]
    ..
    [phrase FOCUS function = Subj
      [word AS samesubject3]
    ]
    ..
    [phrase FOCUS function = Cmpl
      [word AS samecomplement3]
    ]
  ]
  [clause domain = "N"]* {0-1}
  [clause domain = "Q"]* {1-50}
  [clause domain = "N"
    [phrase function = Pred
      [word
        [word lex = "DBR["]
        OR
        [word lex = ">MR["]
        OR
        [word lex = "QR>["]
      ]
    ]
    ..
    [phrase FOCUS function = Cmpl
      [word lex = samecomplement3.lex]
    ]
    ..
    [phrase FOCUS function = Subj
      [word lex = samesubject3.lex]
    ]
  ]]
  OR
  [[clause domain = "N"
    [phrase function = Pred
      [word
        [word lex = "DBR["]
        OR
        [word lex = ">MR["]
        OR
        [word lex = "QR>["]
      ]
    ]
    ..
    [phrase FOCUS function = Cmpl
      [word AS samecomplement4]
    ]
    ..
    [phrase FOCUS function = Subj
      [word AS samesubject4]
    ]
  ]
  [clause domain = "N"]* {0-1}
  [clause domain = "Q"]* {1-50}
  [clause domain = "N"
    [phrase function = Pred
      [word
        [word lex = "DBR["]
        OR
        [word lex = ">MR["]
        OR
        [word lex = "QR>["]
      ]
    ]
    ..
    [phrase FOCUS function = Subj
      [word lex = samesubject4.lex]
    ]
    ..
    [phrase FOCUS function = Cmpl
      [word lex = samecomplement4.lex]
    ]
  ]]
```

In [66]:
(verses, words) = getShebanqData(A, MQL_RESULTS, 9)

344 results in 101 verses with 356 words


This is a complex query. Let's make it simpler first.

We see some recurring objects:


**speakPhrase**

```
[phrase function = Pred
  [word
    [word lex = "DBR["]
      OR
      [word lex = ">MR["]
      OR
      [word lex = "QR>["]
    ]
]
```

**subjPhrase**

```
[phrase FOCUS function = Subj
  [word AS samesubject]
]
```

**cmplPhrase**

```
[phrase FOCUS function = Cmpl
  [word AS samecomplement]
]
```

**clauses**

```
[clause domain = "N"]* {0-1}
[clause domain = "Q"]* {1-50}
```

The structure of the whole query is then, in pseudo TF-query terms:

```
clause domain=N
  speakPhrase
  < s:subjPhrase
  < c:cmplPhrase
clauses
clause domain=N
  speakPhrase
  < subjPhrase (.lex. s)
  < cmplPhrase (.lex. c)
```

OR

```
clause domain=N
  speakPhrase
  < c:cmplPhrase
  < s:subjPhrase
clauses
clause domain=N
  speakPhrase
  < cmplPhrase (.lex. c)
  < subjPhrase (.lex. s)
```

OR

```
clause domain=N
  speakPhrase
  < s:subjPhrase
  < c:cmplPhrase
clauses
clause domain=N
  speakPhrase
  < cmplPhrase (.lex. c)
  < subjPhrase (.lex. s)
```

OR

```
clause domain=N
  speakPhrase
  < c:cmplPhrase
  < s:subjPhrase
clauses
clause domain=N
  speakPhrase
  < subjPhrase (.lex. s)
  < cmplPhrase (.lex. c)
```

The `OR` is only used to enumerate the four different orders between **subjPhrase** and **cmplPhrase**.
So we can simplify greatly!

```
clause domain=N
  sp1:speakPhrase
  < s1:subjPhrase
  c1:cmplPhrase
< clauses
< clause domain=N
  sp2:speakPhrase
  < s2:subjPhrase (.lex. s1)
  c2:cmplPhrase (.lex. c1)
  
sp1 < c1
sp2 < c2
s1 .lex. s2
c1 .lex. c2
```

There is a problem with translating the **clauses**:

```
[clause domain = "N"]* {0-1}
[clause domain = "Q"]* {1-50}
```

The operator `* {n-m}` means: repeat the previous block `n` to `m` times.
In TF-Query we do not have such an operator.

We will mimick this query by means of a mixture of TF-Query and hand-coding.

We examine the results of searching for

```
clause domain=N
  sp: speakPhrase
  < subjPhrase
  c:cmplPhrase
 
sp < c
```

By means of hand coding we walk through the results of this query:

* suppose we are at result `n`
* walk through the following clauses as long as they match **clauses**
* until we meet result `n+k` of the query,
  where result `n` and `n+k` agree lexically in their **subjPhrase** and **cmplPhrase**
* for each such pair of results we add the combination to the result set

We take care to deliver the results in the same way as a TF-query would do.

In [67]:
query = """
clause domain=N
  sp:phrase function=Pred
    word lex=DBR[|>MR[|QR>[
  < phrase function=Subj
  c:phrase function=Cmpl
  
sp < c
"""

In [68]:
speakResults = A.search(query)

  1.38s 1132 results


This is our starting point.

We are going to weave these results together.

The following function does that.
It has to find up to 50 intervening clauses with `domain=Q` between
two clauses with a speech verb.

Let's paramtrize this number 50, so that we can play with it later on.

In [69]:
speakResultsIndex = {sr[0]: sr for sr in speakResults}

def weave(qLimit):
    results = []

    for speakResult in speakResults:
        (clause, speakPhrase, speakWord, subjPhrase, cmplPhrase) = speakResult
        nextClause = L.n(clause, otype="clause")
        if not nextClause:
            continue
        nextClause = nextClause[0]
        domain = F.domain.v(nextClause)
        qSeen = domain == "Q"
        if not qSeen and domain != "N":
            continue
        if not qSeen:
            nextClause = L.n(nextClause, otype="clause")
            if not nextClause:
                continue
            nextClause = nextClause[0]
            domain = F.domain.v(nextClause)
            qSeen = domain == "Q"
        if not qSeen:
            continue
        qs = 1
        while qs <= qLimit:
            nextClause = L.n(nextClause, otype="clause")
            if not nextClause:
                break
            nextClause = nextClause[0]
            domain = F.domain.v(nextClause)
            if domain != "Q":
                break
            qs += 1
        if not nextClause:
            continue
        if domain != "N":
            continue
        if nextClause not in speakResultsIndex:
            continue

        nextSpeakResult = speakResultsIndex[nextClause]
        (nextClause, nextSpeakPhrase, nextSpeakWord, nextSubjPhrase, nextCmplPhrase) = nextSpeakResult

        if (
            {F.lex.v(w) for w in L.d(subjPhrase, otype="word")}
            &
            {F.lex.v(w) for w in L.d(nextSubjPhrase, otype="word")}
        ) and (
            {F.lex.v(w) for w in L.d(cmplPhrase, otype="word")}
            &
            {F.lex.v(w) for w in L.d(nextCmplPhrase, otype="word")}
        ):
            results.append((clause, speakPhrase, subjPhrase, cmplPhrase, nextSubjPhrase, nextCmplPhrase, qs))
        
    print(f"qLimit={qLimit}: {len(results)} results")
    return results

In [70]:
results = weave(50)

qLimit=50: 62 results


In [71]:
(tfVerses, tfWords) = getTfVerses(A, results, (2, 3, 4, 5))

101 verses
356 words


In [72]:
compareResults(A, verses, words, tfVerses, tfWords)

VERSES EQUAL
WORDS EQUAL


What if we allowed only strings of 49 Q-clauses?
Would that matter?

Well, we have sneaked in the number of Q-clauses for each result.
It is added at the end of each result tuple.

Here are the minimum and the maximum that we encountered:

In [73]:
print(f"minimum number of Q-clauses: {min(r[-1] for r in results):>2}")
print(f"maximum number of Q-clauses: {max(r[-1] for r in results):>2}")

minimum number of Q-clauses:  1
maximum number of Q-clauses: 50


So we expect that it does matter if we go from 50 to 49.
Before we test that, let us show all Q-lengths:

In [74]:
for r in results:
    startPhrase = min(r[2], r[3])
    startVerse = T.sectionFromNode(L.u(startPhrase, otype="verse")[0])
    startString = "{} {}:{}".format(*startVerse)
    
    endPhrase = min(r[4], r[5])
    endVerse = T.sectionFromNode(L.u(endPhrase, otype="verse")[0])
    endString = "{} {}:{}".format(*endVerse)
    
    qs = r[-1]
    
    print(f"{startString:<20} == {qs:>2} Q-clauses ==>     {endString}")

Genesis 3:2          ==  7 Q-clauses ==>     Genesis 3:4
Genesis 16:9         ==  2 Q-clauses ==>     Genesis 16:10
Genesis 16:10        ==  2 Q-clauses ==>     Genesis 16:11
Genesis 17:9         == 14 Q-clauses ==>     Genesis 17:15
Genesis 20:9         ==  4 Q-clauses ==>     Genesis 20:10
Genesis 41:38        ==  2 Q-clauses ==>     Genesis 41:39
Genesis 41:39        ==  5 Q-clauses ==>     Genesis 41:41
Exodus 7:14          == 25 Q-clauses ==>     Exodus 7:19
Exodus 7:26          == 12 Q-clauses ==>     Exodus 8:1
Exodus 30:11         == 17 Q-clauses ==>     Exodus 30:17
Exodus 30:17         == 15 Q-clauses ==>     Exodus 30:22
Exodus 30:22         == 24 Q-clauses ==>     Exodus 30:34
Exodus 30:34         == 15 Q-clauses ==>     Exodus 31:1
Leviticus 5:14       == 23 Q-clauses ==>     Leviticus 5:20
Leviticus 5:20       == 25 Q-clauses ==>     Leviticus 6:1
Leviticus 6:1        == 39 Q-clauses ==>     Leviticus 6:12
Leviticus 6:12       == 16 Q-clauses ==>     Leviticus 6:17
Leviti

Let's doublecheck: if we allow only strings of Q-clauses up to length 49, hte result in Leviticus 21:1
should be gone.

In [75]:
results = weave(49)

qLimit=49: 61 results


In [76]:
(tfVerses, tfWords) = getTfVerses(A, results, (2, 3, 4, 5))

 99 verses
350 words


In [77]:
compareResults(A, verses, words, tfVerses, tfWords)

DIFFERENCE:
('Leviticus', 21, 1)
('Leviticus', 22, 17)
DIFFERENCE:
64969 = J:HW@H03
65656 = J:HW@73H


-1

And so it is!

**Conclusion**

Instead of running a query and obtaining a list of results,
we did a bit of programming and we can get much more than just the results.

That is the power of programming.
But programming is difficult, and mistakes will be made.

TF-Query helps you to find a sweet spot between crafting queries
and writing code.

# Example 10a

[Bas Meeuse: Example 10: NTN + object](https://shebanq.ancient-data.org/hebrew/query?version=2017&id=4416)

```
// The object follows the clause-initial predicate
[clause
  [phrase first function IN (Pred, PreS)
    [word vs = qal AND lex = "NTN["]
  ]
  [phrase not function IN (Objc, Cmpl)] *
  [phrase FOCUS function = Objc]
  NOTEXIST [phrase function IN (Objc, Cmpl)]
]

OR

// The object follows the predicate
[clause
  [phrase first not function IN (Objc, Cmpl)]
  [phrase not function IN (Objc, Cmpl)] *
  [phrase function IN (Pred, PreS)
    [word vs = qal AND lex = "NTN["]
  ]
  [phrase not function IN (Objc, Cmpl)] *
  [phrase FOCUS function = Objc]
  NOTEXIST [phrase function IN (Objc, Cmpl)]
]

OR

// The clause-initial object precedes the predicate
[clause
  [phrase FOCUS first function = Objc]
  [phrase not function IN (Objc, Cmpl)] *
  [phrase function IN (Pred, PreS)
    [word vs = qal AND lex = "NTN["]
  ]
  NOTEXIST [phrase function IN (Objc, Cmpl)]
]

OR

// The object precedes the predicate
[clause
  [phrase first not function IN (Objc, Cmpl)]
  [phrase not function IN (Objc, Cmpl)] *
  [phrase FOCUS function = Objc]
  [phrase not function IN (Objc, Cmpl)] *
  [phrase function IN (Pred, PreS)
    [word vs = qal AND lex = "NTN["]
  ]
  NOTEXIST [phrase function IN (Objc, Cmpl)]
]

OR

// The clause-initial object is a pronominal suffix
[clause
  [phrase first function = PreO
    [word vs = qal AND lex = "NTN["]
  ]
  NOTEXIST [phrase function IN (Objc, Cmpl)]
]

OR

// The object is a pronominal suffix
[clause
  [phrase first not function IN (Objc, Cmpl)]
  [phrase not function IN (Objc, Cmpl)] *
  [phrase function = PreO
    [word vs = qal AND lex = "NTN["]
  ]
  NOTEXIST [phrase function IN (Objc, Cmpl)]
]


```

In [78]:
(verses, words) = getShebanqData(A, MQL_RESULTS, "10a")

232 results in 190 verses with 458 words


First we grasp the structure of this query.

All alternatives have, in one way or another

* **phraseNTN** a predicate phrase with the verb `NTN[` in the `qal`
* **phraseNTNps** a predicate phrase with the verb `NTN[` in the `qal` and a pronominal suffix
* **phrases0** a string of zero or more phrases whose function is not `Objc` and not `Cmpl`
* **phrases1** a string of one or more phrases whose function is not `Objc` and not `Cmpl`
* **phraseObj** an object phrase
* a condition that states that after matching the previous elements, there are no
  more phrases with function `Objc` or `Cmpl`

Now let us state the alternatives in pseudo TF-query terms.
We leave out the condition for now:

**A**

```
clause
  =: phraseNTN
  <: phrases0
  <: phraseObj
```

**B**

```
clause
  =: phrases1
  <: phraseNTN
  <: phrases0
  <: phraseObj
```

**C**

```
clause
  =: phraseObj
  <: phrases0
  <: phraseNTN
```

**D**

```
clause
  =: phrases1
  <: phraseObj
  <: phrases0
  <: phraseNTN
```

**E**

```
clause
  =: phraseNTNps
```

**F**

```
clause
  =: phrases1
  <: phraseNTNps
```
  

**N.B. The results that follow E and F do not have a FOCUS block, so nothing of these results
will show up in SHEBANQ.**

While that is probably not what is meant, when we compute the words and verses of the results in TF,
we will leave these out as well.

We can combine **A** and **B** and we can do the same with **C** and **D**, and also **E** and **F**:

**AB**

```
clause
  =: phrases0
  <: phraseNTN
  <: phrases0
  <: phraseObj
```

**CD**

```
clause
  =: phrases0
  <: phraseObj
  <: phrases0
  <: phraseNTN
```

**EF**

```
clause
  =: phrases0
  <: phraseNTNps
```

We cannot make a faithful TF-Query out of this, because of the `*` in the block:

```
[phrase not function IN (Objc, Cmpl)] *
```

What we do is: we make a very simple query for clauses
with a **phraseNTN** and a **phraseObj**,
and for each of them we test whether its phrases conform to one
of the patterns **A** .. **F** above.

In [79]:
query1 = """
clause
  phrase function=Pred|PreS
    word vs=qal lex=NTN[
  phrase function=Objc
"""

query2 = """
clause
  phrase function=PreO
    word vs=qal lex=NTN[
"""

In [80]:
resultCandidates1 = A.search(query1)
resultCandidates2 = A.search(query2)

  0.95s 1246 results
  0.64s 243 results


In [81]:
resultCandidates2a = [(*r, None) for r in resultCandidates2]
resultCandidates = sorted(resultCandidates1 + resultCandidates2a)

In [82]:
def getPhrases(cl):
    phrases = L.d(cl, otype="phrase")
    maxPos = 0
    consecutivePhrases = []
    for phrase in phrases:
        words = L.d(phrase, otype="word")
        start = words[0]
        end = words[-1]
        # exclude phrases inside gaps
        if start < maxPos:
            continue
        consecutivePhrases.append(phrase)
        maxPos = end
    return tuple(consecutivePhrases)
    
results = []

objlike = {"Objc", "Cmpl"}

for r in resultCandidates:
    cl = r[0]
    ntn = r[1]
    obj = r[3]
    phrases = getPhrases(cl)
    
    predSeen = False
    objSeen = False
    
    good = True
    
    for p in getPhrases(cl):
        fn = F.function.v(p)
            
        if not predSeen and not objSeen:
            if p == ntn:
                predSeen = True
            elif p == obj:
                objSeen = True
            elif fn in objlike:
                good = False
                break
            else:
                continue
        elif predSeen and not objSeen:
            if p == obj:
                objSeen = True
            elif fn in objlike:
                good = False
                break
            else:
                continue
        elif not predSeen and objSeen:
            if p == ntn:
                predSeen = True
            elif fn in objlike:
                good = False
                break
            else:
                continue
        elif predSeen and objSeen:
            if fn in objlike:
                good = False
                break
            else:
                continue
                
    if not good:
        continue
    
    if not predSeen:
        continue
        
    pfn = F.function.v(ntn)
    
    if objSeen:
        if pfn == "PreO":
            continue
    else:
        if pfn != "PreO":
            continue
            
    results.append((cl, ntn, obj))
    
print(f"{len(results)} results")

232 results


In [83]:
(tfVerses, tfWords) = getTfVerses(A, results, (2,))

190 verses
458 words


In [84]:
compareResults(A, verses, words, tfVerses, tfWords)

VERSES EQUAL
WORDS EQUAL


We want a closer look at the results, but we have None values for the object phrase in some results.
If we replace them by the first word of the predicate in that same result, we can show the table.

In [85]:
resultso = tuple((cl, pred, L.d(pred, otype="word")[0] if obj is None else obj) for (cl, pred, obj) in results)

In [86]:
A.table(resultso, end=5)

n,p,clause,phrase,phrase
1,Genesis 15:10,וַיִּתֵּ֥ן אִישׁ־בִּתְרֹ֖ו,יִּתֵּ֥ן,אִישׁ־בִּתְרֹ֖ו
2,Genesis 20:6,עַל־כֵּ֥ן לֹא־נְתַתִּ֖יךָ,נְתַתִּ֖יךָ,נְתַתִּ֖יךָ
3,Genesis 23:13,נָתַ֜תִּי כֶּ֤סֶף הַשָּׂדֶה֙,נָתַ֜תִּי,כֶּ֤סֶף הַשָּׂדֶה֙
4,Genesis 30:18,נָתַ֤ן אֱלֹהִים֙ שְׂכָרִ֔י,נָתַ֤ן,שְׂכָרִ֔י
5,Genesis 30:26,תְּנָ֞ה אֶת־נָשַׁ֣י וְאֶת־יְלָדַ֗י,תְּנָ֞ה,אֶת־נָשַׁ֣י וְאֶת־יְלָדַ֗י


Or even closer:

In [87]:
A.show(resultso, end=5, condenseType="clause")

**Conclusion**

This was not an easy one to do by hand-coding.

But the good thing is that once we have obtained the results, we can inspect them really closely.

# Example 10b

[Bas Meeuse: Example 10: NTN + object + recipient](https://shebanq.ancient-data.org/hebrew/query?version=2017&id=4437)

```
[clause focus
  [UNORDEREDGROUP
    [phrase typ = VP AND NOT function IN (PreO, PtcO)
      [word vs = qal AND lex = "NTN["]
    ]
    [phrase function = Objc]
    [phrase function = Cmpl
      [word first lex = "L"]
    ]
  ]
]
OR
[clause focus
  [UNORDEREDGROUP
    [phrase function IN (PreO, PtcO)
      [word vs = qal AND lex = "NTN["]
    ]
    [phrase function = Cmpl
      [word first lex = "L"]
    ]
  ]
]
```

In [88]:
(verses, words) = getShebanqData(A, MQL_RESULTS, "10b")

2087 results in 675 verses with 5573 words


**TO BE DONE**

In [94]:
query = """
"""

In [95]:
# results = A.search(query)

In [96]:
# (tfVerses, tfWords) = getTfVerses(A, results, (2, 4))

In [97]:
# compareResults(A, verses, words, tfVerses, tfWords)